In [ ]:
from manten.data.dataset_maniskill import ManiSkillDataset

cameras = ["camera1", "gripper1"]
# cameras = ["camera1"]
dataset = ManiSkillDataset(
    simulated_length=10000000,
    test_ratio=0.05,
    task="PegInsertionSide-v1",
    # task="PickCube-v1",
    pack_root="/home/i53/student/yagmurlu/code/manten/data/maniskill2/packed_demos",
    obs_horizon=2,
    pred_horizon=16,
    obs_mode="pointcloud",
    state_modality_keys=["tcp_pose"],
    rgb_modality_keys=cameras,
    control_mode="pd_ee_target_delta_pose",
    # control_mode="pd_ee_delta_pose",
    use_mmap=True,
    # use_mmap=False,
    rotation_transform="rotation_6d",
    load_count=2,
)

# print(dataset[0])

dataset_info = dataset.get_dataset_info()

print(dataset_info)

In [ ]:
from torch.utils.data import DataLoader

# Create a dataloader with batch size 4
batch_size = 4
dataloader = DataLoader(
    dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True
)

# Get a sample batch
sample_batch = next(iter(dataloader))

In [ ]:
import einops

obs = sample_batch["observations"]
state_obs = obs["state_obs"]
pcd_obs = obs["pcd_obs"]
pcd_mask = obs["pcd_mask"]
rgb_obs = obs["rgb_obs"]

n_cam = len(pcd_obs)
B, obs_h, C, H, W = next(iter(pcd_obs.values())).shape
pcd_obs = einops.rearrange(list(pcd_obs.values()), "cam b obs_h c h w -> b obs_h cam c h w")
pcd_mask = einops.rearrange(list(pcd_mask.values()), "cam b obs_h c h w -> b obs_h cam c h w")
rgb_obs = einops.rearrange(list(rgb_obs.values()), "cam b obs_h c h w -> b obs_h cam c h w")

# for now only use last observation
pcd_obs = pcd_obs[:, -1]
pcd_mask = pcd_mask[:, -1]
rgb_obs = rgb_obs[:, -1]
state_obs = state_obs[:, -1]

In [ ]:
# encoder:
# _target_: manten.agents.three_dda.encoder.Encoder
# backbone: "clip"
# image_size: [256, 256]
# embedding_dim: ${..._embedding_dim}
# num_sampling_level: 1
# nhist: ${..._num_history}
# num_vis_ins_attn_layers: 2
# fps_subsampling_factor: 3

from manten.networks.vendor.three_dda.encoder import Encoder

# encoder = Encoder(
#     backbone="clip",
#     image_size=(256, 256),
#     embedding_dim=512,
#     num_sampling_level=1,
#     nhist=2,
#     num_vis_ins_attn_layers=2,
#     fps_subsampling_factor=3,
# )
encoder = Encoder(num_sampling_level=1)

In [ ]:
rgb_feats_pyramid, pcd_pyramid, mask_pyramid = encoder.encode_images(
    rgb_obs, pcd_obs, pcd_mask
)

In [ ]:
rpm_l = einops.rearrange(
    [
        rgb_feats_pyramid[0][..., :3],
        pcd_pyramid[0],
        mask_pyramid[0].expand(pcd_pyramid[0].shape),
    ],
    "t b (nc h w) c -> b (nc h) (t w) c",
    nc=2,
    h=16,
    w=16,
)
0